https://yganalyst.github.io/spatial_analysis/uber_h3_keplergl/
https://dailyheumsi.tistory.com/92
https://brorica.tistory.com/148?category=949090
https://thlee33.medium.com/kepler-gl%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EA%B1%B4%EB%AC%BC-3d-%EC%8B%9C%EA%B0%81%ED%99%94-330400887fe3
https://dailyheumsi.tistory.com/147

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
from config import vworld_key
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

In [2]:
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])

In [3]:
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

In [4]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

### 청주시_도시재생계획구역

In [5]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/27.청주시_도시재생계획구역.geojson', encoding='utf-8') as geojson_file:
    geojson_data = json.load(geojson_file)
crp_df = pd.json_normalize(geojson_data) # city_revitalize_planning_map_df
crp_df['geometry'] = crp_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)

# 폴리곤의 중점을 찾음
centroid = crp_df['geometry'].iloc[0].centroid.buffer(0.015) # 1도의 위도 변화는 대략 111.32 킬로미터
crp_df['centroid_polygon_geometry'] = [centroid]
crp_df

,type,properties.fid,properties.구역명,geometry.type,geometry.coordinates,geometry,centroid_polygon_geometry
0,Feature,5,도시재생구역도,Polygon,"[[[127.4833417, 36.6318644], [127.4833417, 36....","POLYGON ((127.4833417 36.6318644, 127.4833417 ...",POLYGON ((127.50347965416363 36.63603755594320...


### 격자(매핑용)

In [6]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/26.청주시_격자(매핑용).geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))

In [7]:
# crp_df에서 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
polygon_expanded = crp_df['centroid_polygon_geometry'].iloc[0]

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')
#grid_map_df['geometry'] = GeoSeries(grid_map_df['geometry'])

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid = grid_map_df[grid_map_df['geometry'].intersects(polygon)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택, within 은 포함
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid_expanded = grid_map_df[grid_map_df['geometry'].intersects(polygon_expanded)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택

### 청주시 성연령별 유동인구

In [99]:
# 원도심 영역(도시재생 대상지역)
floating_population_df = pd.read_csv('SBJ_2309_001/2.청주시_성연령별_유동인구.csv')
floating_population_df['STD_YM'] = floating_population_df['STD_YM'].astype(str)
floating_population_df.rename(columns={'STD_YM':'연도'}, inplace=True)
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
floating_population_df = geo_transform(floating_population_df)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# floating_population_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_floating_population = floating_population_df[floating_population_df['geometry'].within(polygon)]

# 성별 통합한 세대 column 생성
filtered_floating_population['MZ'] = filtered_floating_population.iloc[:, 1:4].sum(axis=1).copy() + filtered_floating_population.iloc[:, 7:10].sum(axis=1).copy()
filtered_floating_population['N_MZ'] = filtered_floating_population.iloc[:, 4:7].sum(axis=1).copy() + filtered_floating_population.iloc[:, 10:13].sum(axis=1).copy()

# 특정 열만 추출하여 성별을 통합한 세대별 유동인구 df생성
filtered_floating_population_gen = filtered_floating_population[list(filtered_floating_population.columns[:1]) + list(filtered_floating_population.columns[16:18]) + [filtered_floating_population.columns[15]]]

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [101]:
# 최종저장용 df 정의 -> 각 지점들을 grid폴리곤에 매핑시켜 grid별 유동인구로 변환 -> 
filtered_floating_population_totgen = pd.DataFrame()
for grid in tqdm(filtered_grid['geometry'].unique()):
    # 격자와 매핑하기 위해 격자 내에 있는 지점으로만 필터링
    subset = filtered_floating_population_gen[filtered_floating_population_gen['geometry'].within(grid)]
    # 해당 구역 grid df 설정(접합용) 
    grid_df = pd.DataFrame({'geometry' : [grid]})
    # 중간저장용 df 정의
    concat_df = pd.DataFrame()
    
    for date in filtered_floating_population_gen['연도'].unique():
        date_df = pd.DataFrame({'연도' : [date[:4]]}) # 해당 연도 df 설정(접합용) 
        subset_date = subset[subset['연도'] == date]
        # 열 별로 더하기 및 df 변환
        column_sums = subset_date.iloc[:,1:3].sum()
        column_sums_df = pd.DataFrame(column_sums).transpose()
        # 최종 df 접합 및 filtered_floating_population_totgen에 concat
        column_sums_df = pd.concat([date_df, column_sums_df], axis=1)
        concat_df = pd.concat([concat_df, column_sums_df])
    else:
        # 각 연도 별로 평균계산 및 df 변환 -> 월별 유동인구를 평균내었기 때문에, 최종값은 해당 격자의 월별 평균 유동인구가 됨.
        column_means_2020 = concat_df[concat_df['연도'].str.startswith('2020')].iloc[:,1:3].mean()
        date_df_2020 = pd.DataFrame({'연도' : ['2020']})
        column_means_2020_df = pd.DataFrame(column_means_2020).transpose()
        tot_df_2020 = pd.concat([date_df_2020, column_means_2020_df, grid_df], axis=1)
        
        column_means_2021 = concat_df[concat_df['연도'].str.startswith('2021')].iloc[:,1:3].mean()
        date_df_2021 = pd.DataFrame({'연도' : ['2021']})
        column_means_2021_df = pd.DataFrame(column_means_2021).transpose()
        tot_df_2021 = pd.concat([date_df_2021, column_means_2021_df, grid_df], axis=1)
        
        column_means_2022 = concat_df[concat_df['연도'].str.startswith('2022')].iloc[:,1:3].mean()
        date_df_2022 = pd.DataFrame({'연도' : ['2022']})
        column_means_2022_df = pd.DataFrame(column_means_2022).transpose()
        tot_df_2022 = pd.concat([date_df_2022, column_means_2022_df, grid_df], axis=1)
        
        tot_df = pd.concat([tot_df_2020, tot_df_2021, tot_df_2022])
        filtered_floating_population_totgen = pd.concat([filtered_floating_population_totgen, tot_df])

filtered_floating_population_totgen

100%|██████████| 156/156 [00:23<00:00,  6.69it/s]


,연도,MZ,N_MZ,geometry
0,2020,31.897500,79.245000,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2021,23.554167,63.031667,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2022,26.054167,77.063333,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2020,588.386667,1078.526667,POLYGON ((127.48881291670402 36.64430329023720...
0,2021,730.630833,1277.312500,POLYGON ((127.48881291670402 36.64430329023720...
...,...,...,...,...
0,2021,50.055000,87.478333,POLYGON ((127.49105137605241 36.63528850233756...
0,2022,63.792500,119.714167,POLYGON ((127.49105137605241 36.63528850233756...
0,2020,16.146667,30.440833,POLYGON ((127.48433936092893 36.63799230366292...
0,2021,13.730833,25.674167,POLYGON ((127.48433936092893 36.63799230366292...


#### 유동인구 HEATMAP(MZ세대 & 비MZ세대)

In [120]:
# map 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2020 = folium.FeatureGroup(name="2020년", overlay=False)
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)

year_lst = ['2020', '2021', '2022']
layers_lst = [m_2020, m_2021, m_2022]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2020부터 2022까지의 연도
    filtered_floating_population_totgen_year = filtered_floating_population_totgen[filtered_floating_population_totgen['연도'] == year].copy()
    # MZ세대 유동인구 정규화
    max_val = filtered_floating_population_totgen_year['MZ'].max()
    filtered_floating_population_totgen_year['MZ'] = filtered_floating_population_totgen_year['MZ'].apply(lambda x : round((x/max_val)*100,2))
    # 'geometry' 열의 값을 위치 정보로 사용하여 MZ세대 유동량 HeatMap 생성
    heat_data = [[point.centroid.xy[1][0], point.centroid.xy[0][0], floating] for point, floating in zip(filtered_floating_population_totgen_year['geometry'], filtered_floating_population_totgen_year['MZ'])]
    HeatMap(heat_data, max_opacity=0.6, radius=25).add_to(layers_dic[year])
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [122]:
# map 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2020 = folium.FeatureGroup(name="2020년", overlay=False)
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)

year_lst = ['2020', '2021', '2022']
layers_lst = [m_2020, m_2021, m_2022]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2020부터 2022까지의 연도
    filtered_floating_population_totgen_year = filtered_floating_population_totgen[filtered_floating_population_totgen['연도'] == year].copy()
    # 비 MZ세대 유동인구 정규화
    max_val = filtered_floating_population_totgen_year['N_MZ'].max()
    filtered_floating_population_totgen_year['N_MZ'] = filtered_floating_population_totgen_year['N_MZ'].apply(lambda x : round((x/max_val)*100,2))
    # 'geometry' 열의 값을 위치 정보로 사용하여 N_MZ세대 유동량 HeatMap 생성
    heat_data = [[point.centroid.xy[1][0], point.centroid.xy[0][0], floating] for point, floating in zip(filtered_floating_population_totgen_year['geometry'], filtered_floating_population_totgen_year['N_MZ'])]
    HeatMap(heat_data, max_opacity=0.6, radius=25).add_to(layers_dic[year])
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

### 청주시 시장 현황 (폴리곤 영역확장 없이 범위 내 시장들만 필터링)

In [8]:
market_df = pd.read_csv('SBJ_2309_001/6.청주시_시장현황.csv')
market_df

,market_nm,addr,area,lon,lat
0,육거리종합시장,상당구 청남로2197번길 42(석교동),86273,127.488184,36.627938
1,서문시장,상당구 남사로89번길 57(서문동),10285,127.485365,36.634894
2,중앙시장,상당구 중앙로 26(북문로2가),7305,127.489192,36.639385
3,사직시장,서원구 사직대로265번길 7(사직동),9200,127.476868,36.636143
4,북부시장,청원구 향군로31번길 19(우암동),22133,127.486221,36.647973


In [9]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
market_df = geo_transform(market_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_market = market_df[market_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



In [12]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_market.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 주차장 현황 (폴리곤 영역확장 고민해봐야 할듯)

In [13]:
parking_df = pd.read_csv('SBJ_2309_001/7.청주시_주차장현황.csv')
parking_df

,carpark_nm,carpark_type,addr,slots,lon,lat
0,청주랜드 동물원,노상,충청북도 청주시 상당구 명암로 171,255,127.515747,36.650530
1,상공회의소,노상,충청북도 청주시 상당구 상당로 102,25,127.490487,36.637486
2,명문식당,노상,충청북도 청주시 상당구 우암산로4,30,127.490514,36.639075
3,고려삼계탕,노상,충청북도 청주시 상당구 교동로3번길 48-1,29,127.490837,36.639529
4,메리츠화재,노상,충청북도 청주시 상당구 교동로3번길 127,20,127.490239,36.643059
...,...,...,...,...,...,...
400,견인차보관소,노외,충청북도 청주시 청원구 각리 641-15,193,127.434307,36.705982
401,공한지주차장,노외,양청리 683-3,18,127.434645,36.722995
402,공한지주차장,노외,내덕동 392-14,4,127.488246,36.661356
403,공한지주차장,노외,정하동 35-3,8,127.464401,36.672168


In [14]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
parking_df = geo_transform(parking_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_parking = parking_df[parking_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



In [15]:
filtered_points_parking # 노상 : 도로 위 한켠에 주차를 하고 일정 비용을 받음(정부 또는 지자체 관할), 노외 : 공용주차장처럼 별도 대지에 설치되어 제공되는 주차장

,carpark_nm,carpark_type,addr,slots,lon,lat,geometry
0,상공회의소,노상,충청북도 청주시 상당구 상당로 102,25,127.490487,36.637486,POINT (127.49049 36.63749)
1,명문식당,노상,충청북도 청주시 상당구 우암산로4,30,127.490514,36.639075,POINT (127.49051 36.63908)
2,고려삼계탕,노상,충청북도 청주시 상당구 교동로3번길 48-1,29,127.490837,36.639529,POINT (127.49084 36.63953)
3,메리츠화재,노상,충청북도 청주시 상당구 교동로3번길 127,20,127.490239,36.643059,POINT (127.49024 36.64306)
4,교서로(지하상가~주성초교),노상,충청북도 청주시 상당구 교서로 45,49,127.485254,36.641427,POINT (127.48525 36.64143)
5,제일학원 앞,노상,충청북도 청주시 상당구 교동로3번길 153,26,127.490198,36.644232,POINT (127.49020 36.64423)
6,시청후문,노상,충청북도 청주시 상당구 상당로143번길 16,26,127.487858,36.641205,POINT (127.48786 36.64121)
7,청주병원,노상,충청북도 청주시 상당구 중앙로 83,35,127.487734,36.644668,POINT (127.48773 36.64467)
8,한전아파트앞,노상,충청북도 청주시 상당구 사직대로361번길 113,28,127.486806,36.641929,POINT (127.48681 36.64193)
9,청중후문앞,노상,충청북도 청주시 상당구 사직대로361번길 125,46,127.485956,36.642952,POINT (127.48596 36.64295)


In [16]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_parking.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 공원 현황 (폴리곤 영역확장 고민해봐야 할듯)

In [20]:
park_df = pd.read_csv('SBJ_2309_001/8.청주시_공원현황.csv')
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
park_df = geo_transform(park_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_park = park_df[park_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



In [21]:
filtered_points_park

,park_gbn,park_nm,lon,lat,geometry
0,소공원,(구)여성회관 앞,127.489598,36.640050,POINT (127.48960 36.64005)
1,어린이공원,남주,127.483619,36.631123,POINT (127.48362 36.63112)
2,테마공원,청주중앙공원,127.487601,36.632739,POINT (127.48760 36.63274)
3,근린공원,상당공원,127.491402,36.636940,POINT (127.49140 36.63694)
4,광장,청주청소년광장,127.488435,36.639388,POINT (127.48843 36.63939)
5,근린공원,옛청주역사공원,127.488923,36.640869,POINT (127.48892 36.64087)


In [22]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_park.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 공공기관 및 주요지점 현황

In [31]:
public_df = pd.read_csv('SBJ_2309_001/25.청주시_공공기관_및_주요지점현황.csv')
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
public_df = geo_transform(public_df)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



In [32]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_public = public_df[public_df['geometry'].within(polygon)].reset_index(drop=True)

In [34]:
filtered_points_public

,inst_gbn,inst_nm,lon,lat,geometry
0,시군청,청주우체국,127.488969,36.633807,POINT (127.48897 36.63381)
1,시군청,중앙동주민센터,127.488198,36.639856,POINT (127.48820 36.63986)
2,시군청,청주시의회,127.489673,36.641613,POINT (127.48967 36.64161)
3,금융기관,우리은행 청주지점,127.487831,36.631849,POINT (127.48783 36.63185)
4,금융기관,신한은행 청주지점,127.489080,36.631181,POINT (127.48908 36.63118)
5,금융기관,수협중앙회 청주지점,127.490309,36.639388,POINT (127.49031 36.63939)
6,금융기관,농협 충북지역본부,127.489332,36.643592,POINT (127.48933 36.64359)
7,금융기관,외환은행 청주지점,127.491314,36.631163,POINT (127.49131 36.63116)
8,금융기관,한국산업은행 청주지점,127.488861,36.635399,POINT (127.48886 36.63540)
9,공공기관,한국감정원 청주지점,127.489525,36.634582,POINT (127.48952 36.63458)


In [35]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_public.iterrows():
    popup_text = f"{row['inst_gbn']} - {row['inst_nm']}"
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m).add_child(folium.Popup(popup_text, max_width=200))

# 지도 저장
m

### 청주시 가로수 현황 (공공데이터포털 - 산림청_도시숲가로수관리 가로수 현황 data https://www.data.go.kr/data/15120900/fileData.do)

In [ ]:
'''산림청_도시숲가로수관리 가로수 현황_20221231 데이터 정제 완료 -'''
# street_tree_df = pd.read_csv('SBJ_2309_001/산림청_도시숲가로수관리 가로수 현황_20221231.csv', encoding='cp949')
# street_tree_df = street_tree_df[['시군구명', '도로구간명', '수종명', '지역X좌표', '지역Y좌표']]
# street_tree_df = street_tree_df[street_tree_df['시군구명'].str.contains('청주')].reset_index(drop=True)

# # EPSG 5186에서 EPSG 4326으로 좌표 변환을 수행하는 함수 생성
# def transform_coordinates(row):
#     in_proj = pyproj.Proj(init='epsg:5186')
#     out_proj = pyproj.Proj(init='epsg:4326')
#     lon, lat = pyproj.transform(in_proj, out_proj, row['지역X좌표'], row['지역Y좌표'])
#     return pd.Series({'lat': lat, 'lon': lon})

# # '지역X좌표'와 '지역Y좌표'를 EPSG 4326으로 변환하고 'lat' 및 'lon' 열로 이름 변경
# street_tree_df[['lat', 'lon']] = street_tree_df.apply(transform_coordinates, axis=1)
# # '지역X좌표'와 '지역Y좌표' 열을 삭제 (선택 사항)
# street_tree_df.drop(['지역X좌표', '지역Y좌표'], axis=1, inplace=True)

# 

'산림청_도시숲가로수관리 가로수 현황_20221231 데이터 정제 완료 -'

In [36]:
street_tree_df = pd.read_csv('SBJ_2309_001/29.청주시_가로수현황.csv', encoding='UTF8')
street_tree_df

,시군구명,도로구간명,수종명,lat,lon
0,충청북도 청주시 청원구,오창대로,벚나무,36.708975,127.481089
1,충청북도 청주시 청원구,오창대로,벚나무,36.708883,127.480707
2,충청북도 청주시 청원구,오창대로,벚나무,36.708996,127.481787
3,충청북도 청주시 청원구,오창대로,벚나무,36.708625,127.480314
4,충청북도 청주시 청원구,오창대로,벚나무,36.708993,127.481158
...,...,...,...,...,...
24481,충청북도 청주시 청원구,율중로29번길,이팝나무,36.660599,127.507634
24482,충청북도 청주시 청원구,율중로29번길,이팝나무,36.660668,127.507612
24483,충청북도 청주시 청원구,율중로29번길,이팝나무,36.660737,127.507589
24484,충청북도 청주시 청원구,율중로29번길,이팝나무,36.660249,127.507748


In [37]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
street_tree_df = geo_transform(street_tree_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_tree = street_tree_df[street_tree_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



In [38]:
filtered_points_tree

,시군구명,도로구간명,수종명,lat,lon,geometry
0,충청북도 청주시 상당구,교서로,은행나무,36.637265,127.486389,POINT (127.48639 36.63726)
1,충청북도 청주시 상당구,남사로,은행나무,36.632017,127.487616,POINT (127.48762 36.63202)
2,충청북도 청주시 상당구,남사로,은행나무,36.632060,127.489971,POINT (127.48997 36.63206)
3,충청북도 청주시 상당구,남사로,은행나무,36.632081,127.488882,POINT (127.48888 36.63208)
4,충청북도 청주시 상당구,남사로,은행나무,36.632098,127.489197,POINT (127.48920 36.63210)
...,...,...,...,...,...,...
454,충청북도 청주시 상당구,상당로70번길,칠엽수,36.634603,127.491657,POINT (127.49166 36.63460)
455,충청북도 청주시 상당구,상당로70번길,칠엽수,36.634618,127.491736,POINT (127.49174 36.63462)
456,충청북도 청주시 상당구,상당로81번길,버즘나무,36.635674,127.490370,POINT (127.49037 36.63567)
457,충청북도 청주시 상당구,상당로81번길,버즘나무,36.635540,127.490390,POINT (127.49039 36.63554)


In [39]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_tree.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 불법주정차

In [ ]:
illegal_parking_df = pd.read_csv('SBJ_2309_001/14.청주시_불법주정차단속통계.csv')
illegal_parking_df = geo_transform(illegal_parking_df)
illegal_parking_df

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,cd_type,cd_loc,cd_dong,cd_cnt,lon,lat,geometry
0,고정형,상당로 남궁요양병원사거리부근1,문화동,13,127.490864,36.632164,POINT (127.49086 36.63216)
1,고정형,사직대로 대현지하상가나도약국부근,북문로1가,15,127.487698,36.636797,POINT (127.48770 36.63680)
2,고정형,사직대로 지하상가성안길입구부근,북문로2가,17,127.488849,36.636973,POINT (127.48885 36.63697)
3,고정형,상당로 도청사거리부근,북문로1가,20,127.490597,36.634411,POINT (127.49060 36.63441)
4,고정형,무심동로 육거리주차장부근,석교동,20,127.487675,36.627044,POINT (127.48768 36.62704)
5,고정형,청소년광장사거리부근,북문로2가,272,127.488643,36.639136,POINT (127.48864 36.63914)
6,고정형,홈플러스후문사거리부근,서문동,374,127.488256,36.635447,POINT (127.48826 36.63545)
7,고정형,상당로81번길 산업은행청주점부근,북문로1가,133,127.488747,36.635504,POINT (127.48875 36.63550)
8,고정형,사직대로350번길 청주상호저축은행부근,서문동,129,127.486397,36.634354,POINT (127.48640 36.63435)
9,고정형,교서로 청주공고부근,영동,52,127.486449,36.637435,POINT (127.48645 36.63744)


In [ ]:
# map 생성
m = folium.Map(location=[36.627797, 127.511943],  zoom_start=12)

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

for _, row in illegal_parking_df.iterrows() :
    popup_text = f"{row['cd_loc']}({row['cd_dong']} - {row['cd_cnt']}건)"
    folium.Circle(location=(row['lat'], row['lon']), radius=30, color='#FF580B',
            fill='#FF580B').add_to(m).add_child(folium.Popup(popup_text, max_width=200))
    
m

### 청주시 112신고 통계 (격자매핑 형식)

In [ ]:
police_report_df = pd.read_csv('SBJ_2309_001/20.청주시_112신고통계.csv')
# 5개년 신고건수 총합
police_report_df['repo_total'] = police_report_df['repo_2018'] + police_report_df['repo_2019'] + police_report_df['repo_2020'] + police_report_df['repo_2021'] + police_report_df['repo_2022']
police_report_df['repo_total'] = police_report_df['repo_total'].astype(int)
police_report_df = police_report_df[['gid', 'repo_total']]

#원도심 영역 grid id 리스트 로드 및 police_report_df 필터링
grid_id = filtered_grid['properties.gid'].tolist()
filtered_police_report = police_report_df[police_report_df['gid'].isin(grid_id)]
filtered_police_report['geometry'] = filtered_grid['geometry'].tolist()
filtered_police_report.sort_values(by='repo_total', ascending=False, inplace=True)
filtered_police_report.reset_index(drop=True, inplace=True)
filtered_police_report

C:\Users\user\AppData\Local\Temp\ipykernel_6272\1370399623.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_police_report['geometry'] = filtered_grid['geometry'].tolist()
C:\Users\user\AppData\Local\Temp\ipykernel_6272\1370399623.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_police_report.sort_values(by='repo_total', ascending=False, inplace=True)


,gid,repo_total,geometry
0,다바989482,1519,POLYGON ((127.48769621544852 36.63168220843648...
1,다바988483,1510,"POLYGON ((127.4865775332584 36.63258358652596,..."
2,다바992477,1406,POLYGON ((127.49105231410148 36.62717501134686...
3,다바991478,953,POLYGON ((127.48993373614363 36.62807642164767...
4,다바990486,887,POLYGON ((127.4888142201003 36.635288313298695...
...,...,...,...
268,다바994489,2,"POLYGON ((127.49328829744827 36.637993143918, ..."
269,다바996484,2,POLYGON ((127.49552579225157 36.63348575733207...
270,다바984494,2,POLYGON ((127.48210108412849 36.64249947656094...
271,다바995485,1,POLYGON ((127.49440717516781 36.63438720867091...


#### 경찰신고수는 평균에 대해 얼마나 높은가를 구해서 시각화 해야할듯...?

KeplerGl로 시각화 해보기

In [ ]:
len(filtered_police_report)

273

In [ ]:
# 데이터프레임에서 데이터 추출
lst = filtered_police_report['repo_total']

# 조건에 따라 각 막대의 색상을 설정
colors = ['green' if x > 600 else 'yellow' if x > 300 else 'red' for x in lst]

# 히스토그램을 생성
fig = go.Figure(go.Histogram(x=lst, marker_color=colors, nbinsx=1000))

# 레이아웃 설정
fig.update_xaxes(title_text='report_count')
fig.update_yaxes(title_text='count')
fig.update_layout(title_text='report_count_histogram')

# 플롯리 그래프 보이기
fig.show()

In [ ]:
# map 생성
m = folium.Map(location=[36.627797, 127.511943],  zoom_start=12)

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# 신고건수에 따라 색상을 지정하는 함수
def color_producer(repo_total):
    if repo_total < 300:
        return 'green'
    elif 300 <= repo_total < 800:
        return 'yellow'
    else:
        return 'red'
        
for _, row in filtered_police_report.iterrows() :
    popup_text = f"격자:{row['gid']}(신고건수 - {row['repo_total']}건)"
    color = color_producer(row['repo_total'])
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 0.2}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
m

### 청주시 카드매출 (격자매핑) 다시 확인해야 할듯?

KB국민카드 데이터를 활용하기 보다는 총 매출 추정금액을 활용하는 것이 더 효율적일 것으로 예상됨

In [35]:
cardsales_df = pd.read_csv('SBJ_2309_001/19.청주시_카드매출_격자매핑.csv')
cardsales_df.dropna(inplace=True)
cardsales_df.reset_index(drop=True, inplace=True)
cardsales_df['est_sales'] = (cardsales_df['est_sales'] * 1000).astype(int)
cardsales_df
#원도심 영역 grid id 리스트 로드 및 police_report_df 필터링
grid_id = filtered_grid['properties.gid'].tolist()
filtered_cardsales = cardsales_df[cardsales_df['gid'].isin(grid_id)]
# 'est_sales' 열의 값이 0인 행을 'card_sales' 값으로 채우기
filtered_cardsales['est_sales'] = filtered_cardsales.apply(lambda row: row['card_sales'] if row['est_sales'] == 0 else row['est_sales'], axis=1)
filtered_cardsales

C:\Users\user\AppData\Local\Temp\ipykernel_9444\4040690819.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,stdr_ym,kbc_bzc_nm_1,kbc_bzc_nm_2,kbc_bzc_nm_3,card_sales,est_sales,gid
0,202305,서비스,(정기)납부/대여서비스,미디어/음향,989000,18995000,다바989487
1,202305,서비스,(정기)납부/대여서비스,도서/음반대여,54000,844000,다바989487
2,202305,서비스,의료서비스,한의원,5019200,83333000,다바992490
3,202305,소매업,의류,여성의류,1600000,25979000,다바990484
4,202305,소매업,패션잡화,구두제화,103500,3315000,다바989483
...,...,...,...,...,...,...,...
13006,202306,소매업,의류,일반의류(기타),1657400,1657400,다바990481
13007,202305,음식,커피/음료,커피전문점,98200,98200,다바987489
13008,202306,소매업,패션잡화,액세서리,109010,109010,다바988486
13009,202305,음식,기타외국식,쌀국수전문점,8000,8000,다바991482


In [36]:
geometry_lst = []
for _, row in filtered_cardsales.iterrows():
    geometry_lst.append(map_dic[row['gid']])
filtered_cardsales['geometry'] = geometry_lst

C:\Users\user\AppData\Local\Temp\ipykernel_9444\1955721771.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [66]:
filtered_cardsales[filtered_cardsales['stdr_ym'] == 202306]

,stdr_ym,kbc_bzc_nm_1,kbc_bzc_nm_2,kbc_bzc_nm_3,card_sales,est_sales,gid,geometry
2165,202306,음식,간이주점,호프/맥주,25000,375000,다바992481,POLYGON ((127.49105189723711 36.63078100870997...
2166,202306,소매업,의류,일반의류(기타),358800,11778000,다바988484,POLYGON ((127.48657737690299 36.63348508523775...
2169,202306,소매업,패션잡화,잡화점,1444000,15570000,다바988486,POLYGON ((127.48657706417131 36.63528808225117...
2171,202306,서비스,생활편의서비스,간판/광고,1452000,37084000,다바989489,POLYGON ((127.48769521207878 36.63799269753248...
2172,202306,소매업,패션잡화,잡화점,328000,5659000,다바987481,POLYGON ((127.48545933314207 36.63078045743661...
2174,202306,소매업,가전제품,종합가전제품,133000,14172000,다바991486,POLYGON ((127.48993279807374 36.63528841306920...
2175,202306,소매업,의류,일반의류(기타),4284700,41387000,다바988488,POLYGON ((127.48657675141185 36.63709107871767...
2176,202306,음식,커피/음료,커피전문점,128000,1071000,다바990480,POLYGON ((127.48881500185995 36.62987932057243...
2177,202306,음식,양식,종합양식,1126800,18374000,다바993487,POLYGON ((127.49216986281625 36.63619007942166...
2179,202306,소매업,의류,일반의류(기타),7196100,61205000,다바990481,POLYGON ((127.48881487158114 36.63078081970196...


In [ ]:
#  매출 액수에 따라 색상을 지정하는 함수
def color_picker(est_sales_sum):
    if est_sales_sum > 2.500900e+07:
        return 'red'
    elif 2.500900e+07 > est_sales_sum >= 1.050500e+07:
        return 'orange'
    elif 1.050500e+07 > est_sales_sum >= 3.580000e+06:
        return 'yellow'
    elif 3.580000e+06 > est_sales_sum >= 0:
        return 'green'
        
# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 100px; height: 110px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 0.8);
     "> &nbsp; 총매출 범례 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 상위 25% <br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 상위 50% <br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 상위 75% <br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 상위 100%
     </div>
     """
     
# map 생성
m = folium.Map(location=[36.627797, 127.511943],  zoom_start=13)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_1 = folium.FeatureGroup(name="1월", overlay=False) 
m_2 = folium.FeatureGroup(name="2월", overlay=False)
m_3 = folium.FeatureGroup(name="3월", overlay=False)
m_4 = folium.FeatureGroup(name="4월", overlay=False)
m_5 = folium.FeatureGroup(name="5월", overlay=False)
m_6 = folium.FeatureGroup(name="6월", overlay=False)

month_lst = [202301, 202302, 202303, 202304, 202305, 202306]
layers_lst = [m_1, m_2, m_3, m_4, m_5, m_6]
layers_dic = {}
for month, layer in zip(month_lst, layers_lst):
    layers_dic[month] = layer

for month in filtered_cardsales['stdr_ym'].unique():
    filtered_cardsales_month = filtered_cardsales[filtered_cardsales['stdr_ym'] == month]
    # 'gid' 값을 기준으로 그룹화
    for gid in filtered_cardsales_month['gid'].unique():
        subset = filtered_cardsales_month[filtered_cardsales_month['gid'] == gid].copy()
        subset['kbc_total'] = subset.iloc[:, 1] + ' - ' +  subset.iloc[:, 3]
        est_sales_sum = subset['est_sales'].sum()
        
        # 매출 데이터를 3자리마다 쉼표로 구분하고 "원"을 추가
        subset['카테고리'] = subset['kbc_total'].copy()
        subset['총매출'] = subset['est_sales'].apply(lambda x: f'{x:,}원')
            
        # subset_popup 데이터프레임을 HTML로 변환하여 popup text에 저장
        subset_popup = subset[['카테고리', '총매출']].style.hide(axis='index').set_properties(**{'border': '1px solid black'}).to_html()
        popup_text = f"총매출 - {est_sales_sum:,}원<br><br><div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{subset_popup}</div>"
        color = color_picker(est_sales_sum)
        folium.GeoJson(
            filtered_cardsales_month[filtered_cardsales_month['gid'] == gid]['geometry'].iloc[0],
            style_function=lambda feature, color=color: {
                'fillColor': color,
                'color': 'black',
                'weight': 0.5}
        ).add_to(layers_dic[month]).add_child(folium.Popup(popup_text, max_width=1200))

for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)
# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))

m

### 청주시 도로명주소(건물 + 도로)

In [14]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/15.청주시_도로명주소(건물).geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_building_df = pd.json_normalize(geojson_data['features'])
roadname_building_df = roadname_building_df[['properties.BD_MGT_SN', 'properties.RN_CD', 'properties.GRO_FLO_CO', 'properties.UND_FLO_CO', 'geometry.coordinates']]
roadname_building_df['properties.GRO_FLO_CO'] = roadname_building_df['properties.GRO_FLO_CO'].astype(int)
roadname_building_df['properties.UND_FLO_CO'] = roadname_building_df['properties.UND_FLO_CO'].astype(int)
roadname_building_df['geometry'] = roadname_building_df['geometry.coordinates'].apply(lambda x : make_pol(x))

In [15]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
roadname_building_df = gpd.GeoDataFrame(roadname_building_df, geometry='geometry')

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_roadname_building = roadname_building_df[roadname_building_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_roadname_building

,properties.BD_MGT_SN,properties.RN_CD,properties.GRO_FLO_CO,properties.UND_FLO_CO,geometry.coordinates,geometry
0,4311110100100600000061755,3236004,5,0,"[[[127.48694447846057, 36.6400016341025], [127...","POLYGON ((127.48694 36.64000, 127.48694 36.639..."
1,4311110100100550002032236,3236004,1,0,"[[[127.48632680823786, 36.64052917039099], [12...","POLYGON ((127.48633 36.64053, 127.48633 36.640..."
2,4311110100100560003032233,3236004,2,0,"[[[127.48655786043167, 36.640535159309565], [1...","POLYGON ((127.48656 36.64054, 127.48646 36.640..."
3,4311110100100560003071386,3236004,1,0,"[[[127.48653912832565, 36.640421876670125], [1...","POLYGON ((127.48654 36.64042, 127.48646 36.640..."
4,4311110100100560002032235,3236004,3,0,"[[[127.48657086929198, 36.64041777756533], [12...","POLYGON ((127.48657 36.64042, 127.48656 36.640..."
...,...,...,...,...,...,...
3184,4311110800100300004000001,4508315,4,0,"[[[127.49253748307129, 36.63019961400344], [12...","POLYGON ((127.49254 36.63020, 127.49279 36.630..."
3185,4311110100100040003046395,3236024,1,0,"[[[127.48581567013639, 36.644730907282785], [1...","POLYGON ((127.48582 36.64473, 127.48588 36.644..."
3186,4311110800100590001049620,4508315,5,0,"[[[127.49155463874942, 36.63035531760507], [12...","POLYGON ((127.49155 36.63036, 127.49155 36.630..."
3187,4311110900101650001000001,4508240,3,0,"[[[127.48496189891529, 36.63447898239693], [12...","POLYGON ((127.48496 36.63448, 127.48514 36.634..."


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_roadname_building.iterrows():
    popup_text = f"지상:{row['properties.GRO_FLO_CO']}층, 지하:{row['properties.UND_FLO_CO']}층)"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'color': 'black', 'weight': 1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

https://thlee33.medium.com/kepler-gl%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EA%B1%B4%EB%AC%BC-3d-%EC%8B%9C%EA%B0%81%ED%99%94-330400887fe3

In [16]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/16.청주시_도로명주소(도로).geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_road_df = pd.json_normalize(geojson_data['features'])
roadname_road_df['geometry'] = roadname_road_df['geometry.coordinates'].apply(lambda x: LineString(x))
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadname_road_df = gpd.GeoDataFrame(roadname_road_df, geometry='geometry')

In [17]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_roadname_road = roadname_road_df[roadname_road_df['geometry'].within(polygon)].reset_index(drop=True)

In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_roadname_road.iterrows():
    popup_text = f"도로명:{row['properties.RN']}"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'color': 'black', 'weight': 5, 'opacity': 0.7}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

### 청주시 인도(보도) 시각화

In [19]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/17.청주시_인도(보도).geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
sidewalk_df = pd.json_normalize(geojson_data['features'])
sidewalk_df['geometry'] = sidewalk_df['geometry.coordinates'].apply(make_lin)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
sidewalk_df = gpd.GeoDataFrame(sidewalk_df, geometry='geometry')

In [32]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0].centroid.buffer(0.015)

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_sidewalk = sidewalk_df[sidewalk_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_sidewalk

,type,properties.UFID,properties.QUAL,properties.BYYN,properties.KIND,geometry.type,geometry.coordinates,geometry
0,Feature,100036706050A0031fb2bb5069e324009,미분류,무,인도,LineString,"[[127.48704805856802, 36.64879715261194], [127...","LINESTRING (127.48705 36.64880, 127.48703 36.6..."
1,Feature,100036706040A0031944d0191c32c411d,미분류,무,인도,LineString,"[[127.4905110148085, 36.65017086233968], [127....","LINESTRING (127.49051 36.65017, 127.49050 36.6..."
2,Feature,100036706050A0031b5e7f9c289f64455,미분류,무,인도,LineString,"[[127.48811663566829, 36.645383249505215], [12...","LINESTRING (127.48812 36.64538, 127.48809 36.6..."
3,Feature,100036706050A00319fde6787305a4ceb,미분류,무,인도,MultiLineString,"[[[127.4821726136577, 36.646733069397726], [12...","LINESTRING (127.48217 36.64673, 127.48202 36.6..."
4,Feature,100036706050A003171957b43200a469d,미분류,무,인도,LineString,"[[127.4892521501864, 36.64684313842911], [127....","LINESTRING (127.48925 36.64684, 127.48907 36.6..."
...,...,...,...,...,...,...,...,...
624,Feature,100036706060A00318412b193c1854672,미분류,유,자전거도로,LineString,"[[127.49352268082293, 36.62464723956152], [127...","LINESTRING (127.49352 36.62465, 127.49340 36.6..."
625,Feature,100036706060A00319220d74c063e442a,블록,유,자전거도로,LineString,"[[127.4940249394998, 36.624428771048855], [127...","LINESTRING (127.49402 36.62443, 127.49389 36.6..."
626,Feature,100036706060A003119b74a925b834b89,블록,유,자전거도로,LineString,"[[127.49415527886981, 36.62422537476052], [127...","LINESTRING (127.49416 36.62423, 127.49405 36.6..."
627,Feature,100036706060A0031ca0403c6c49b4b58,블록,유,자전거도로,LineString,"[[127.4895393709331, 36.62559462233704], [127....","LINESTRING (127.48954 36.62559, 127.48970 36.6..."


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_sidewalk.iterrows():
    popup_text = f"도로명:{row['properties.KIND']}"
    color = 'black' if row['properties.KIND'] == '인도' else 'blue'
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color=color: {'fillColor': color, 'color': color, 'weight': 2}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

### 청주시 상권정보 시각화

In [71]:
trading_area_df = pd.read_csv('SBJ_2309_001/18.청주시_상권정보.csv')
trading_area_df = geo_transform(trading_area_df)
trading_area_df

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



,com_lc_cd,com_lc_nm,com_mc_cd,com_mc_nm,com_sc_cd,com_sc_nm,ksic_cd,ksic_nm,adb_emd_cd,adb_emd_nm,road_address,lon,lat,geometry
0,R,학문/교육,R04,학원-어학,R04A01,학원-외국어/어학,P85502,외국어학원,4311259000,성화.개신.죽림동,"충청북도 청주시 서원구 복대로 7, (개신동)",127.446061,36.617928,POINT (127.44606 36.61793)
1,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,비알콜 음료점업,4311154500,성안동,"충청북도 청주시 상당구 대성로 38, (서운동)",127.493371,36.628602,POINT (127.49337 36.62860)
2,D,소매,D07,가정/주방/인테리어,D07A15,유리/페인트/철물건축자재,G47519,"페인트, 유리 및 기타 건설자재 소매업",4311257000,수곡1동,"충청북도 청주시 서원구 매봉로 64, (수곡동, 산남주공3차)",127.478662,36.616083,POINT (127.47866 36.61608)
3,D,소매,D15,가구소매,D15A01,일반가구소매,G47520,가구 소매업,4311231000,남이면,"충청북도 청주시 서원구 남이면 청남로 1272, (척산리)",127.440092,36.562169,POINT (127.44009 36.56217)
4,R,학문/교육,R04,학원-어학,R04A01,학원-외국어/어학,P85502,외국어학원,4311454000,율량.사천동,"충청북도 청주시 청원구 공항로 118, (율량동)",127.485764,36.667772,POINT (127.48576 36.66777)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46118,L,부동산,L01,부동산중개,L01A01,부동산중개,L68221,부동산 자문 및 중개업,4311454000,율량.사천동,"충청북도 청주시 청원구 사뜸로61번길 106, (사천동, 대창5차)",127.476366,36.664730,POINT (127.47637 36.66473)
46119,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,4311425300,오창읍,"충청북도 청주시 청원구 오창읍 두릉유리로 1130-15, (장대리)",127.455312,36.741021,POINT (127.45531 36.74102)
46120,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,4311154500,성안동,"충청북도 청주시 상당구 무심동로390번길 15, (서문동)",127.484757,36.634341,POINT (127.48476 36.63434)
46121,D,소매,D02,선물/팬시/기념품,D02A02,꽃집/꽃배달,G47851,화초 및 산식물 소매업,4311374700,가경동,"충청북도 청주시 흥덕구 풍년로198번길 56-1, (가경동)",127.432312,36.631408,POINT (127.43231 36.63141)


['음식', '소매', '생활서비스', '학문/교육', '부동산', '관광/여가/오락', '숙박', '스포츠'],

In [ ]:
# a = filtered_trading_area[filtered_trading_area['com_lc_nm'] == '소매']
# a['com_mc_nm'].unique()

In [ ]:
# filtered_trading_area['com_lc_nm'].value_counts()

In [ ]:
# a['com_mc_nm'].value_counts()

In [72]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_trading_area = trading_area_df[trading_area_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_trading_area

,com_lc_cd,com_lc_nm,com_mc_cd,com_mc_nm,com_sc_cd,com_sc_nm,ksic_cd,ksic_nm,adb_emd_cd,adb_emd_nm,road_address,lon,lat,geometry
0,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,기타 주점업,4311154500,성안동,"충청북도 청주시 상당구 남사로140번길 6, (남문로2가)",127.490409,36.631851,POINT (127.49041 36.63185)
1,D,소매,D10,건강/미용식품,D10A07,건강원,G47216,건강보조식품 소매업,4311154500,성안동,"충청북도 청주시 상당구 상당로3번길 18-1, (석교동)",127.490050,36.628551,POINT (127.49005 36.62855)
2,D,소매,D05,의복의류,D05A10,한복/갈옷/민속옷,G47412,한복 소매업,4311154500,성안동,"충청북도 청주시 상당구 무심동로336번길 100-2, (남문로1가)",127.488456,36.631302,POINT (127.48846 36.63130)
3,D,소매,D05,의복의류,D05A02,캐쥬얼/스포츠의류,G47416,셔츠 및 기타 의복 소매업,4311154500,성안동,"충청북도 청주시 상당구 상당로 55, (남문로2가)",127.490343,36.633284,POINT (127.49034 36.63328)
4,D,소매,D07,가정/주방/인테리어,D07A02,주방용품,G47592,식탁 및 주방용품 소매업,4311154500,성안동,"충청북도 청주시 상당구 상당로1번길 22, (석교동)",127.489694,36.628361,POINT (127.48969 36.62836)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,D,소매,D23,자동차/자동차용품,D23A11,자동차유리전문점,G45219,기타 자동차신품 부품 및 내장품 판매업,4311152500,중앙동,"충청북도 청주시 상당구 사북로 138, (영동)",127.486152,36.644819,POINT (127.48615 36.64482)
3174,F,생활서비스,F05,인력/고용/용역알선,F05A02,인력공급/고용알선,NaN,NaN,4311154500,성안동,"충청북도 청주시 상당구 용담로 7, (문화동, 문화동센트럴칸타빌)",127.492115,36.632598,POINT (127.49211 36.63260)
3175,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,4311154500,성안동,"충청북도 청주시 상당구 사직대로362번길 46, (서문동)",127.487471,36.634714,POINT (127.48747 36.63471)
3176,D,소매,D16,화장품소매,D16A01,화장품판매점,G47813,화장품 및 방향제 소매업,4311152500,중앙동,"충청북도 청주시 상당구 상당로 127, (북문로2가)",127.489698,36.639536,POINT (127.48970 36.63954)


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_trading_area.iterrows():
    popup_text = f"{row['com_lc_nm']}-{row['com_mc_nm']}-{row['com_sc_nm']}"
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

### 청주시 주민등록인구 현황

In [24]:
pop_df = pd.read_csv('SBJ_2309_001/22.청주시_주민등록인구현황.csv')
pop_df['tot_pop'] = pop_df.iloc[:,3:].sum(axis=1)
target = ['충청북도 청주시 상당구 성안동', '충청북도 청주시 상당구 중앙동']
pop_df_1 = pop_df[pop_df['adm_nm'] == (target[0])]
pop_df_1 = pop_df_1[['year','tot_pop']].reset_index(drop=True)
pop_df_1.rename(columns = {'tot_pop' : '성안동'}, inplace = True)
pop_df_2 = pop_df[pop_df['adm_nm'] == (target[1])]
pop_df_2 = pop_df_2[['year','tot_pop']].reset_index(drop=True)
pop_df_2.rename(columns = {'tot_pop' : '중앙동'}, inplace = True)
tot_pop_df = pd.merge(pop_df_1, pop_df_2)
tot_pop_df['year'] = tot_pop_df['year'].astype(str)
tot_pop_df

,year,성안동,중앙동
0,201701,6401,6508
1,201702,6361,6488
2,201703,6341,6457
3,201704,6341,6483
4,201705,6315,6481
...,...,...,...
67,202208,6720,6867
68,202209,6676,6757
69,202210,6688,6744
70,202211,6678,6740


In [25]:
# tot_pop_df 데이터프레임을 사용하여 꺾은선 그래프를 생성합니다.
fig = px.line(tot_pop_df, x='year', y=['성안동', '중앙동'],
              title='성안동 및 중앙동 인구 데이터',
              labels={'year': '연도', 'value': '인구수'},
              markers=True)

# 그래프를 표시합니다.
fig.show()

### 청주시 건물 노후도

구조별 수명에 대해 노후도 및 위험도 책정

In [26]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/23.청주시_건물노후도.geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)
building_year_df = pd.json_normalize(geojson_data['features'])
building_year_df = building_year_df.dropna(subset=['properties.old_year'])
building_year_df['properties.old_year'] = building_year_df['properties.old_year'].astype(int)
building_year_df['geometry'] = building_year_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
building_year_df = gpd.GeoDataFrame(building_year_df, geometry='geometry')

NameError: name 'make_pol' is not defined

In [ ]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_building_year = building_year_df[building_year_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_building_year

,type,properties.pnu,properties.emd_nm,properties.bld_nm,properties.dong_nm,properties.gfa,properties.strct_nm,properties.usage_nm,properties.useapr_day,properties.old_year,geometry.type,geometry.coordinates,geometry
0,Feature,4311110100100020001,충청북도 청주시 상당구영동,None,None,145.26,벽돌구조,단독주택,1978-01-20,46,MultiPolygon,"[[[[127.48671936764237, 36.644922806739345], [...","POLYGON ((127.48672 36.64492, 127.48673 36.644..."
1,Feature,4311110100100020002,충청북도 청주시 상당구영동,None,None,92.43,경량철골구조,단독주택,2011-04-12,13,MultiPolygon,"[[[[127.48644934729707, 36.644848446412425], [...","POLYGON ((127.48645 36.64485, 127.48657 36.644..."
2,Feature,4311110100100020003,충청북도 청주시 상당구영동,None,주2,173.13,경량철골구조,제2종근린생활시설,2011-09-14,13,MultiPolygon,"[[[[127.48626989607264, 36.644893917411586], [...","POLYGON ((127.48627 36.64489, 127.48641 36.644..."
3,Feature,4311110100100020004,충청북도 청주시 상당구영동,None,None,140.13,벽돌구조,제1종근린생활시설,1978-01-16,46,MultiPolygon,"[[[[127.48673982638863, 36.64474369781375], [1...","POLYGON ((127.48674 36.64474, 127.48667 36.644..."
4,Feature,4311110100100040001,충청북도 청주시 상당구영동,None,None,84.60,경량철골구조,제2종근린생활시설,2001-05-11,23,MultiPolygon,"[[[[127.48620407621114, 36.64486541245635], [1...","POLYGON ((127.48620 36.64487, 127.48620 36.644..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2143,Feature,4311111200104120017,충청북도 청주시 상당구 수동,None,None,199.83,벽돌구조,단독주택,1990-10-18,34,MultiPolygon,"[[[[127.49108479582297, 36.644601827356276], [...","POLYGON ((127.49108 36.64460, 127.49093 36.644..."
2144,Feature,4311111200104120020,충청북도 청주시 상당구 수동,None,None,162.38,철근콘크리트구조,단독주택,2015-11-17,9,MultiPolygon,"[[[[127.49072487086501, 36.64459902566444], [1...","POLYGON ((127.49072 36.64460, 127.49077 36.644..."
2145,Feature,4311111200104120027,충청북도 청주시 상당구 수동,None,None,118.11,벽돌구조,단독주택,1983-12-08,41,MultiPolygon,"[[[[127.49048536397038, 36.64466430392876], [1...","POLYGON ((127.49049 36.64466, 127.49048 36.644..."
2146,Feature,4311111200104120027,충청북도 청주시 상당구 수동,None,None,99.94,벽돌구조,단독주택,1985-12-28,39,MultiPolygon,"[[[[127.490738466091, 36.644671104512796], [12...","POLYGON ((127.49074 36.64467, 127.49074 36.644..."


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_building_year.iterrows():
    popup_text = f"구조명:{row['properties.strct_nm']}, 용도명:{row['properties.usage_nm']}<br>연식:{row['properties.old_year']}년"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'color': 'black', 'weight': 1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

## turtle로 그림그리는 방안 생각해보기?